In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime

import os
import multiprocessing as mp

import threading

In [2]:
def request_platform(obj, platform):
    if(platform == 'amazon'):
        return obj.find_all("div", {"class": "s-result-item"})
    if(platform == 'flipkart'):
        return obj.find_all("div", {"class": "_1AtVbE"})
    if(platform == 'gumtree'):
        return obj.find_all("article")
    
    return 'nothing'

In [3]:
def request(url, platform):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get(url, headers=headers)
    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    items = request_platform(soup2,platform)
    
    return items


In [4]:
def flipkart_scrape(key_word, q, nb = 9):
    key_word = key_word.replace(" ", "%20")
    url = 'https://www.flipkart.com/search?q='+ key_word
    items = request(url, 'flipkart')
    array_flpikart = []
    for i in range(3, 13):
        try:
            array_flpikart.append(flipkart_details(items[i]))
        except:
            pass
    
    flpikart = array_flpikart
    q.put(flpikart)

def flipkart_details(item):
    image = item.find("img", {"class": "_396cs4"})['src']
    link = item.find("a", {"class": "_1fQZEK"})['href']
    link_url = 'https://www.flipkart.com' + link
    
    title = item.find("div", {"class": "_4rR01T"}).get_text().strip()[0:]
    
    description_list = item.find("ul", {"class": "_1xgFaf"}).find_all("li")
    description = ''
    for li in description_list:
        description += li.get_text().strip()[0:] + ', '
    
    price_text = item.find("div", {"class": "_30jeq3 _1_WHN1"}).get_text().strip()[0:]   
    price_amount = price_text[1:]
    price_symbol = price_text[0]
    
    #json
    result = {"image": image, "link_url": link_url, 
              "title": title, "price_text": price_text, "price_amount": price_amount,
              "price_symbol": price_symbol, "description": description}
    
    return result


In [5]:
def gumtree_scrape(key_word, q, nb = 9):
    key_word = key_word.replace(" ", "%20")
    
    url = 'https://www.gumtree.com/search?search_category=all&q='+ key_word
    items = request(url, 'gumtree')
    array_gumtree = []
    for i in range(1, 9):
        array_gumtree.append(gumtree_details(items[i]))
    
    gumtree = array_gumtree
    q.put(gumtree)

def gumtree_details(item):
    image = item.find("img")['src']
    if(image == ''):
        image = item.find("img")['data-src']
      
    link = item.find("a", {"class": "listing-link"})
    link_url = 'https://www.gumtree.com' + link['href']
    
    title = item.find("h2").get_text().strip()[0:] 
    price_text = item.find("span", {"class": "listing-price"}).get_text().strip()[0:]  
    price_amount = price_text[0]
    price_symbol = price_text[1:]
    location = item.find("div", {"class": "listing-location"}).get_text().strip()[0:]  
    description = item.find("p", {"class": "listing-description"}).get_text().strip()[0:]  
    
    #json
    result = {"image": image, "link_url": link_url, 
              "title": title, "price_text": price_text, "price_amount": price_amount,
              "location": location, "description": description}
    
    return result

In [6]:
def amazon_scrape(key_word, q, nb = 9):
    key_word = key_word.replace(" ", "+")
    url = 'https://www.amazon.com/s?k='+ key_word +'&ref=nb_sb_noss_2'
    items = request(url, 'amazon')
    array_amazon = []
    for i in range(1, 9):
        try:
            array_amazon.append(amazon_details(items[i]))
        except:
            pass
    
    amazon = array_amazon
    q.put(amazon)

def amazon_details(item):
    image = item.find("img", { "class" : "s-image" })['src']
    link = item.find("a", {"class": "a-link-normal"})
    link_url = 'https://www.amazon.com' + link['href']
    
    details = item.find("div", {"class": "sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20"})
    title = details.find("a").get_text().strip()[0:]
    price_text = details.find("span", {"class": "a-offscreen"}).get_text().strip()[0:]
    price_amount = details.find("span", {"class": "a-price-whole"}).get_text().strip()[0:] 
    price_symbol = details.find("span", {"class": "a-price-symbol"}).get_text().strip()[0:] 
    ppl_rated = details.find("span", {"class": "a-size-base"}).get_text().strip()[0:] 
    rate_text = details.find("span", {"class": "a-declarative"}).get_text().strip()[0:] 
    
    #json
    result = {"image": image, "link_url": link_url, 
              "title": title, "price_text": price_text, "price_amount": price_amount,
              "price_symbol": price_symbol, "ppl_rated": ppl_rated, rate_text: "rate_text"}
    
    return result


In [7]:
#global variables
amazon = []
gumtree = []
flipkart = []


In [43]:
#Linear execution
key_word = 'lenovo laptop'

starttime = time.time()

#gumtree = gumtree_scrape(key_word)
amazon = amazon_scrape(key_word)
flipkart = flipkart_scrape(key_word)

time.time() - starttime

4.059105634689331

In [9]:
import threading

key_word = 'lenovo laptop'
  
if __name__ == "__main__":
    q1 = mp.Queue()
    q2 = mp.Queue()
    q3 = mp.Queue()
    
    # creating thread
    t1 = threading.Thread(target=amazon_scrape, args=(key_word,q1))
    t2 = threading.Thread(target=flipkart_scrape, args=(key_word,q2))
    t3 = threading.Thread(target=gumtree_scrape, args=(key_word,q3))
  
    starttime = time.time()
    t1.start()
    t2.start()
    t3.start()
  
    t1.join()
    t2.join()
    t3.join()
    duration = time.time() - starttime  
    # both threads completely executed
    
    amazon = q1.get()
    flipkart = q2.get()
    gumtree = q3.get()
    
    print("Done!")    
    print(duration)
    os.kill

Done!
2.5679190158843994
